
# Word Similarity
内容：単語の類似度を測定<br>
word2vecとfastTextでやってみる<br>
実行環境：google colab

# MeCab
途中でMeCabが必要になるので，あらかじめ入れておく

In [91]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null
!echo mecab-config --dicdir"/mecab-ipadic-neologd"

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.
mecab-config --dicdir/mecab-ipadic-neologd


In [0]:
import MeCab
path_NEologd = 'd /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd'

In [97]:
m = MeCab.Tagger('-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
wakati = m.parse('COVID-19のせいで就職氷河期になりそうでぴえんって感じ')
print(wakati) # ちゃんとNEologdが使えていることを確認

COVID-19 の せい で 就職氷河期 に なり そう で ぴえん って 感じ 



In [102]:
m = MeCab.Tagger(path)
result = m.parse('猫')
print(result)

猫	名詞,一般,*,*,*,*,猫,ネコ,ネコ
EOS



# word2vec

In [0]:
from gensim.models import word2vec
import logging

In [0]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colaboratory_file/NLP/neko_wakati.csv')
df.head() # 確認

,raw,preprocessed,wakati_mecab,wakati_sp
0,吾輩《わがはい》は猫である。名前はまだ無い。,吾輩は猫である。名前はまだ無い。,猫 名前 無い,吾輩は猫である 。 名前はまだ 無い 。
1,どこで生れたかとんと見当《けんとう》がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣い...,どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけ...,生れる 見当 つく 薄暗い する 泣く いた事 記憶 する 始める 人間 見る あと 聞く ...,どこ で 生れ た かと んと 見当 が つか ぬ 。 何でも 薄暗 い じ め じ め ...
2,この書生の掌の裏《うち》でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運...,この書生の掌の裏でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運転し始め...,書生 掌 裏 よい 心持 坐る おる する 非常 速力 運転 する 書生 動く 自分 動く ...,この 書生 の 掌 の 裏 で しばらく は よい 心持 に 坐って おった が 、 しば...
3,ふと気が付いて見ると書生はいない。たくさんおった兄弟が一｜疋《ぴき》も見えぬ。肝心《かんじ...,ふと気が付いて見ると書生はいない。たくさんおった兄弟が一疋も見えぬ。肝心の母親さえ姿を隠し...,気 付く 見る 書生 いる たくさん おる 兄弟 一疋 見える 肝心 母親 姿 隠す 上今 ...,ふと 気が 付いて 見ると 書生 は いない 。 た く さん おった 兄 弟 が一 疋 ...
4,ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろ...,ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろ...,思い 笹原 這い出す 向う 池 ある 池 前 坐る する よい 考える 見る 分別 出る す...,ようやく の 思い で 笹原 を 這 い 出すと 向う に 大きな 池 がある 。 吾輩は...


nanを取り除く作業

In [0]:
wakati_mecab_list = [row.wakati_mecab for index, row in df.iterrows()]
nan_index = [i for i, wakati in enumerate(wakati_mecab_list) if str(wakati) == 'nan']
len(nan_index) # 87個のnanが存在する
for i in sorted(nan_index, reverse = True):
    wakati_mecab_list.pop(i)
# reverse = False だとnan_indexが昇順なので前から取り除かれて失敗しそう
nan_index_new = [i for i, wakati in enumerate(wakati_mecab_list) if str(wakati) == 'nan']
len(nan_index_new) # 確認

0

分かち書きを二重listにする

In [0]:
wakati_mecab_split = [word.split(' ') for word in wakati_mecab_list]

word2vec学習

In [0]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model_trained = word2vec.Word2Vec(wakati_mecab_split,
                         sg=1, # Skip-gram
                         size=200, # 次元数
                         min_count=5, # min_count回未満の単語を破棄
                         window=5, # 文脈の最大単語数
                         hs=1, # 階層ソフトマックス(ネガティブサンプリングするなら0)
                         negative=5, # ネガティブサンプリング
                         iter=50 # Epoch数
                         )

2020-04-19 08:29:47,295 : INFO : collecting all words and their counts
2020-04-19 08:29:47,295 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-04-19 08:29:47,319 : INFO : collected 10083 word types from a corpus of 65937 raw words and 2168 sentences
2020-04-19 08:29:47,320 : INFO : Loading a fresh vocabulary
2020-04-19 08:29:47,335 : INFO : effective_min_count=5 retains 2422 unique words (24% of original 10083, drops 7661)
2020-04-19 08:29:47,337 : INFO : effective_min_count=5 leaves 53142 word corpus (80% of original 65937, drops 12795)
2020-04-19 08:29:47,352 : INFO : deleting the raw counts dictionary of 10083 items
2020-04-19 08:29:47,354 : INFO : sample=0.001 downsamples 38 most-common words
2020-04-19 08:29:47,355 : INFO : downsampling leaves estimated 43586 word corpus (82.0% of prior 53142)
2020-04-19 08:29:47,360 : INFO : constructing a huffman tree from 2422 words
2020-04-19 08:29:47,417 : INFO : built huffman tree with maximum node depth 13
20

word2vecのモデル保存

In [0]:
model_trained.save('/content/drive/My Drive/Colaboratory_file/NLP/neko_w2v_mecab.model')

2020-04-19 08:30:58,801 : INFO : saving Word2Vec object under /content/drive/My Drive/Colaboratory_file/NLP/neko_w2v_mecab.model, separately None
2020-04-19 08:30:58,803 : INFO : not storing attribute vectors_norm
2020-04-19 08:30:58,804 : INFO : not storing attribute cum_table
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-04-19 08:30:58,919 : INFO : saved /content/drive/My Drive/Colaboratory_file/NLP/neko_w2v_mecab.model


word2vecのモデル読み込み

In [0]:
model_w2v = word2vec.Word2Vec.load('/content/drive/My Drive/Colaboratory_file/NLP/neko_w2v_mecab.model')

2020-04-19 08:31:00,962 : INFO : loading Word2Vec object from /content/drive/My Drive/Colaboratory_file/NLP/neko_w2v_mecab.model
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-04-19 08:31:01,050 : INFO : loading wv recursively from /content/drive/My Drive/Colaboratory_file/NLP/neko_w2v_mecab.model.wv.* with mmap=None
2020-04-19 08:31:01,051 : INFO : setting ignored attribute vectors_norm to None
2020-04-19 08:31:01,056 : INFO : loading vocabulary recursively from /content/drive/My Drive/Colaboratory_file/NLP/neko_w2v_mecab.model.vocabulary.* with mmap=None
2020-04-19 08:31:01,057 : INFO : loading trainables recursively from /content/drive/My Drive/Colaboratory_file/NLP/neko_w2v_m

類似度測定

In [119]:
print('=====word2vec trained by MeCab=====')
s = input('類似度の計測をしたい単語を入力：')
try:
    word_low = s.lower()
    m = MeCab.Tagger(path_NEologd)
    word_ori = m.parse(s).split('\t')[1].split(',')[6]
    results = model_w2v.most_similar(positive=[word_ori], topn=20)
    for i, result in enumerate(results):
        print(i+1, '\t', '{0:.5f}'.format(result[1]), ' ', result[0].lower())
except KeyError:
    print('{}が辞書の中に入っていない'.format(s))

=====word2vec trained by MeCab=====
類似度の計測をしたい単語を入力：猫
1 	 0.34151   属
2 	 0.32575   三毛
3 	 0.30698   鳴き声
4 	 0.29930   進化
5 	 0.29760   脳裏
6 	 0.29530   南無阿弥陀仏
7 	 0.29265   条件
8 	 0.28784   主人
9 	 0.27948   思う
10 	 0.27622   あら
11 	 0.27330   疑う
12 	 0.27066   する
13 	 0.27021   人間
14 	 0.26865   感ずる
15 	 0.26556   誇る
16 	 0.26288   下女
17 	 0.26260   同族
18 	 0.25715   生涯
19 	 0.25620   かご
20 	 0.25560   足


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


結果としては，小さいコーパスにしては悪くない気がする．

## 類似度を測りたい語句がいくつかある場合

In [0]:
df_word_list = ['人間', '猫', '無能', '猫達','あいうえお']

word2vecで類似度計測をする関数定義

In [0]:
def w2v_sim(word, num):
    try:
        results = model_w2v.most_similar(positive=[word], topn=num)
        sim_words = [sim_word for sim_word, sim_cos in results]
    except KeyError:
        sim_words = [0]*num
    return(sim_words)

計測して，DataFrameを作成

In [116]:
df_sim = pd.DataFrame()
sim_level = 10
for word in df_word_list:
    df_sim[word] = w2v_sim(word, sim_level)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [117]:
df_sim

,人間,猫,無能,猫達,あいうえお
0,動物,属,全能,0,0
1,陥る,三毛,断定,0,0
2,衣服,鳴き声,上等,0,0
3,珍重,進化,失敗,0,0
4,運命,脳裏,断言,0,0
5,歴史,南無阿弥陀仏,栄,0,0
6,平等,条件,能力,0,0
7,人間らしい,主人,製造,0,0
8,意義,思う,灰色,0,0
9,能力,あら,疑う,0,0


計測できなかった場合は，0を入れておくことにした．<br>
ここで重要なのは猫はできるが，猫達はできないという事．<br>
こうする事で，カラムの消失を避ける事ができる．

# fastText

In [0]:
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!sudo pip install .

Cloning into 'fastText'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 3768 (delta 35), reused 46 (delta 15), pack-reused 3684
Receiving objects: 100% (3768/3768), 8.20 MiB | 5.55 MiB/s, done.
Resolving deltas: 100% (2354/2354), done.
/content/fastText
Processing /content/fastText
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2872951 sha256=784e8a0054b80a470aca033e8f3070db89c619c59969a3f8be19231ca1096758
  Stored in directory: /tmp/pip-ephem-wheel-cache-x3iv8ikh/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext


In [0]:
import fasttext as ft

csvにしないと上手くできなかった．<br>
理由はよくわからない．．．

In [0]:
df_wakati_mecab = pd.DataFrame(wakati_mecab_list, columns=['wakati'])
df_wakati_mecab.head()

In [0]:
path = '/content/drive/My Drive/Colaboratory_file/NLP/neko_wakati_mecab.csv'
df_wakati_mecab.to_csv(path, header=True, index=False)

In [0]:
model_trained = ft.train_unsupervised(path,
                                      minn=2, # Ngram最小文字
                                      maxn=8, # Ngram最大文字
                                      dim=200, # 次元
                                      epoch=50, # エポック
                                      lr=0.1, # 学習率
                                      thread=2) # 学習の並列処理数

In [0]:
model_trained.save_model('/content/drive/My Drive/Colaboratory_file/NLP/neko_ft_mecab.bin')

In [108]:
model_ft = ft.load_model('/content/drive/My Drive/Colaboratory_file/NLP/neko_ft_mecab.bin')

In [120]:
print('=====fasttext trained by MeCab=====')
s = input('類似度の計測をしたい単語を入力：')
word_low = s.lower()
m = MeCab.Tagger(path_NEologd)
word_ori = m.parse(s).split('\t')[1].split(',')[6]
results = model_ft.get_nearest_neighbors(word_ori, k=20)
for i, result in enumerate(results):
    print(i+1, '\t', '{0:.5f}'.format(result[0]), ' ', result[1].lower())

=====fasttext trained by MeCab=====
類似度の計測をしたい単語を入力：猫
1 	 0.36584   人間
2 	 0.34531   する
3 	 0.32547   南無阿弥陀仏
4 	 0.32055   一疋
5 	 0.31664   ない
6 	 0.31560   動物
7 	 0.30543   進化
8 	 0.30425   思う
9 	 0.29995   見る
10 	 0.29411   鼠
11 	 0.28909   称する
12 	 0.28769   称す
13 	 0.28652   なる
14 	 0.27900   銭
15 	 0.27104   同族
16 	 0.27010   能力
17 	 0.26355   相互
18 	 0.26167   重んずる
19 	 0.25985   庸
20 	 0.25935   光景


## 類似度を測りたい語句がいくつかある場合

In [0]:
df_word_list = ['人間', '猫', '無能', '猫達','あいうえお']

fastTextを用いた類似度計測の関数定義

In [0]:
def ft_sim(word, num):
    results = model_ft.get_nearest_neighbors(word, k=num)
    sim_words = [sim_word for sim_cos, sim_word in results]
    return(sim_words)

In [0]:
df_sim = pd.DataFrame()
sim_level = 10
for word in df_word_list:
    df_sim[word] = ft_sim(word, sim_level)

In [124]:
df_sim

,人間,猫,無能,猫達,あいうえお
0,する,人間,全能,友達,あがる
1,ない,する,断言,発達,あら
2,猫,南無阿弥陀仏,証明,猫,あきらめる
3,出来る,一疋,あつかう,つらい,あした
4,云う,ない,断定,ただ今,呼吸
5,意義,動物,眼前,母堂,あばた
6,なる,進化,畠,活気,魚
7,無能,思う,手際,一疋,柿
8,ある,見る,勘弁,不道徳,あける
9,世の中,鼠,深夜,風邪,あく


word2vecとは異なり，計測できないことが存在しない．<br>
word2vecではできなかった猫達でもそれなりに測れている．<br>
しかし，あいうえおは無理やり測った感が強いので，何も見ないと騙されそう．